In [1]:
!pip install --upgrade --quiet pip
!pip install --quiet farm-haystack[colab,faiss]==1.17.2
print('pip install haystack complete.')

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
chex 0.1.82 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.52 which is incompatible.
kfp 2.0.1 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.23.5 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scip

In [2]:
URL = 'https://yplus.ps/wp-content/uploads/2021/01/Said-Edward-Out-of-Place-A-Memoir-1.pdf'
WORKING = '/kaggle/working/'

import os
import requests

# https://stackoverflow.com/a/56951135
def download(url: str, dest_folder: str):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)  # create folder if it does not exist

    filename = url.split('/')[-1].replace(" ", "_")  # be careful with file names
    file_path = os.path.join(dest_folder, filename)

    r = requests.get(url, stream=True)
    if r.ok:
        print("saving to", os.path.abspath(file_path))
        with open(file=file_path, mode='wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 8):
                if chunk:
                    f.write(chunk)
                    f.flush()
                    os.fsync(f.fileno())
    else:  # HTTP status code 4XX/5XX
        print("Download failed: status code {}\n{}".format(r.status_code, r.text))


short_name = os.path.basename(URL)
if not os.path.exists(WORKING + short_name):
    download(url=URL, dest_folder=WORKING)
    print('downloaded {} to {}'.format(URL, WORKING))
else:
    print('file {} already exists.'.format(WORKING + short_name))
pdf_paths=[WORKING + short_name]

saving to /kaggle/working/Said-Edward-Out-of-Place-A-Memoir-1.pdf
downloaded https://yplus.ps/wp-content/uploads/2021/01/Said-Edward-Out-of-Place-A-Memoir-1.pdf to /kaggle/working/


In [3]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(embedding_dim=128, faiss_index_factory_str='Flat')
print('created our document store')

created our document store


In [4]:
!apt-get install --quiet poppler-utils -y
print('pip install poppler utils complete.')
!sudo apt-get update
!sudo apt-get install --quiet poppler-utils -y
print('pip install poppler complete for real and for true')

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libpoppler97 poppler-data
Suggested packages:
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  libpoppler97 poppler-data poppler-utils
0 upgraded, 3 newly installed, 0 to remove and 40 not upgraded.
Need to get 2562 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 poppler-data all 0.4.9-2 [1475 kB]
Ign:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler97 amd64 0.86.1-0ubuntu1.3
Ign:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.3
Err:2 http://security.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler97 amd64 0.86.1-0ubuntu1.3
  404  Not

In [5]:
# Initialize a PDFToTextConverter to convert the PDF to text
from haystack.utils import convert_files_to_docs, fetch_archive_from_http, clean_wiki_text
from haystack.utils import fetch_archive_from_http, convert_files_to_docs
from haystack.nodes import PDFToTextConverter, PreProcessor, DensePassageRetriever
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
print('many imports complete.')

many imports complete.


In [6]:
#replace this paths with your input Data path

# Convert the PDF documents to Haystack-compatible format
pdf_converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=['en'])
pdf_docs = []

for pdf_path in pdf_paths:
    doc = pdf_converter.convert(file_path=pdf_path, meta=None)[0]
    pdf_docs.append(doc)
print('converted {} documents'.format(len(pdf_docs)))

pdftotext version 0.86.1
Copyright 2005-2020 The Poppler Developers - http://poppler.freedesktop.org
Copyright 1996-2011 Glyph & Cog, LLC


converted 1 documents


In [7]:
# Initializing the PreProcessor to clean and split the text
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by='word',
    split_length=100,
    split_respect_sentence_boundary=True,
)
processed_docs = preprocessor.process(pdf_docs)
print('document processing done.')

Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

document processing done.


## Initializing the Retriever

We use a `DensePassageRetriever` and we invoke `update_embeddings` to index the embeddings of documents in the `FAISSDocumentStore`.

In [8]:
# Initialize a DensePassageRetriever
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='vblagoje/dpr-question_encoder-single-lfqa-wiki',
    passage_embedding_model='vblagoje/dpr-ctx_encoder-single-lfqa-wiki',
)
print('built retriever.')

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


built retriever.


In [9]:
document_store.write_documents(processed_docs)
document_store.update_embeddings(retriever)
print('updated embeddings.')

Writing Documents:   0%|          | 0/1557 [00:00<?, ?it/s]

Updating Embedding:   0%|          | 0/1557 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/1568 [00:00<?, ? Docs/s]

updated embeddings.


In [10]:
from haystack.nodes import Seq2SeqGenerator

generator = Seq2SeqGenerator(model_name_or_path='vblagoje/bart_lfqa')
print('built generator.')

built generator.


In [11]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path='deepset/deberta-v3-large-squad2')
print('built the reader.')

built the reader.


In [12]:
from haystack.pipelines import GenerativeQAPipeline
from haystack.pipelines import ExtractiveQAPipeline

generative_pipeline = GenerativeQAPipeline(generator, retriever)
extractive_pipeline = ExtractiveQAPipeline(reader, retriever)

In [13]:
query = 'What is this book about?'
result = extractive_pipeline.run(query=query, params={'Retriever': {'top_k': 5}, 'Reader': {'top_k': 1}})
print(result)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

{'query': 'What is this book about?', 'no_ans_gap': 15.176605939865112, 'answers': [<Answer {'answer': 'Out of Place', 'type': 'extractive', 'score': 0.7543542385101318, 'context': 'y of friends or\ninstitutions in France and Egypt. I began to work on Out of Place in May 1994\nwhile I was recovering from three early rounds of chemot', 'offsets_in_document': [{'start': 525, 'end': 537}], 'offsets_in_context': [{'start': 69, 'end': 81}], 'document_ids': ['6b2d1124e049981443e6003a55238d3b'], 'meta': {'_split_id': 7, 'vector_id': '613'}}>], 'documents': [<Document: {'content': 'Said’s book is about the mysterious … inner power that\nenables some people to overcome the stunting in uences of their childhoods\nand stubbornly keep growing toward a light, at rst dimly and intermittently\nglimpsed, that is their destiny to reach.”\n—Forward\x0cEDWARD W. SAID\nOut of Place\nEdward W. Said is University Professor of English and Comparative Literature\nat Columbia University. He is the author of tw

In [14]:
print(result['answers'][0].answer)

Out of Place


In [15]:
piperun=generative_pipeline.run(
    query='What is this book about?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

I don't know much about the book, but I can tell you that it's a very interesting read. It's about a man named Edward Said. He's a professor of English and Comparative Literature at Columbia University, and he wrote a book called *Out of Place*. It's a memoir of his childhood, and it's very interesting.


In [16]:
piperun=generative_pipeline.run(
    query='Who was Edward Said?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

Edward the Confessor was an Anglo-Saxon king who ruled England from the 10th century to the 11th century. He was the son of Edward I of England and his first wife, Anne Boleyn. Edward's father, Edward II, was the King of England from 1066 to 1137. Edward was the father of Edward III of England, Edward IV of Scotland, and Edward V of England. Edward II was the first king of England to be crowned King of the United Kingdom. Edward III's son, Edward V, became King of Great Britain and Ireland from 1137 to 1141. Edward V was the last king to rule England from 1141 to 1142. Edward VI died in 1141, and his son Edward III succeeded him as King of Britain.


In [17]:
piperun=generative_pipeline.run(
    query='Where did Edward Said go to school?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

I'm not sure if this is what you're looking for, but Edward Said did go to school in the United States. He was born in New York City, and was educated at the New York State School for the Deaf, which was a boarding school for deaf children.


In [18]:
piperun=generative_pipeline.run(
    query='What countries are mentioned in this book?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

I can't speak for the rest of the book, but I can tell you that there are a couple of countries that are mentioned in the book. The first is the United Kingdom, and the second is the Ottoman Empire. The Ottoman Empire was the largest empire in the world at the time of Said's writing, with a population of about 2.5 million.


In [19]:
piperun=generative_pipeline.run(
    query='What are the major themes of this book?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

I'm not a historian, but I've read a lot of Said's work, and I can tell you that he's a very good writer. I'm not sure what you're looking for, though, so I'll try to give you a general overview of the book. The book is about a man named Edward Said, who was born and raised in the United States. He grew up in New York City, and went on to study at Columbia University, where he became a professor of English and Comparative Literature. He wrote a number of books, including *The Orientalist*, *Culture and Imperialism, Representations of the Intellectual, and The Politics of EssenceDispossession*, and *Out of Place*. His work is very well-researched and well-written, and it's easy to see why he's considered one of the most important historians of the 20th century. He's a great writer, and his work has a lot to do with


In [20]:
piperun=generative_pipeline.run(
    query='Please tell me everything about Edward Said.', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

Edward Said was one of the most influential intellectuals of the 20th century. He wrote a number of books on the history of the Middle East, including *The Origins of the Modern World* and *The Rise and Fall of the Third Reich*. He also wrote a book called *A People's History of the World* which is a collection of essays written by people from all over the world.


In [21]:
piperun=generative_pipeline.run(
    query='What did Edward Said think of the Balfour Declaration?', params={'Retriever': {'top_k': 2}} 
)
for answer in piperun['answers']:
    print(answer.answer)

The Balfour Declaration was signed by the Prime Minister of the United Kingdom, Neville Chamberlain, on November 1, 1947. It was the first step in a series of policies that would eventually lead to the creation of the modern state of Israel.


In [22]:
piperun=generative_pipeline.run(
    query='Where was Edward Said in 1991?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

Edward Said was a well-known journalist in the late 1980s and early 1990s. He was a member of the International Committee of the Red Cross (ICRC) and wrote a number of books on the subject of human rights and human rights abuses in the Middle East. In 1991, he was the head of the ICRC's Office for Research and Analysis of Human Rights, which was responsible for collecting and analyzing human rights data from around the world.


In [23]:
piperun=generative_pipeline.run(
    query='Did the committee conclude that the CIA detention and interrogation program was ethical?', params={'Retriever': {'top_k': 5}} #output varies with different top_k values
)
for answer in piperun['answers']:
    print(answer.answer)

No, the committee did not conclude that the CIA detention and interrogation program was ethical.


In [24]:
piperun=generative_pipeline.run(
    query='What is Edward Said\'s legacy?', params={'Retriever': {'top_k': 5}}
)
piperun['answers'][0].answer

'Edward Said is one of the most important historians of the 20th century. He wrote a number of books on the history of the Middle East, including *The Origins of the Modern World* and *The Rise and Fall of the Third Reich*. He also wrote a book called *The Making of Modern Europe* which is a collection of essays written by people from all over the world about the development of modern Europe.'